# Data labeling and human-in-the-loop pipelines with Amazon Augmented AI (A2I)

### Introduction

In this lab you will create your own human workforce, a human task UI, and then define the human review workflow to perform data labeling. You will make the original predictions of the labels with the custom ML model, and then create a human loop if the probability scores are lower than the preset threshold. After the completion of the human loop tasks, you will review the results and prepare data for re-training.

### Table of Contents

- [1. Set up Amazon Cognito user pool and define human workforce](#c3w3-1.)
  - [1.1. Create Amazon Cognito user pool](#c3w3-1.1.)
    - [Exercise 1](#c3w3-ex-1)
  - [1.2. Create Amazon Cognito user pool client](#c3w3-1.2.)
    - [Exercise 2](#c3w3-ex-2)
  - [1.3. Create Amazon Cognito user pool domain and group](#c3w3-1.3.)
    - [Exercise 3](#c3w3-ex-3)
    - [Exercise 4](#c3w3-ex-4)
  - [1.4. Create workforce and workteam](#c3w3-1.4.)
    - [Exercise 5](#c3w3-ex-5)
    - [Exercise 6](#c3w3-ex-6)
  - [1.5. Create an Amazon Cognito user and add the user to the group](#c3w3-1.5.)
- [2. Create Human Task UI](#c3w3-2.)
  - [Exercise 7](#c3w3-ex-7)
- [3. Define human review workflow](#c3w3-3.)
  - [Exercise 8](#c3w3-ex-8)
- [4. Start human loop with custom ML model](#c3w3-4.)
  - [4.1. Deploy a custom model](#c3w3-4.1.)
    - [Exercise 9](#c3w3-ex-9)
  - [4.2. Start the human loop](#c3w3-4.2.)
    - [Exercise 10](#c3w3-ex-10)
  - [4.3. Check status of the human loop](#c3w3-4.3.)
  - [4.4. Complete the human loop tasks](#c3w3-4.4.)
  - [4.5. Verify that the human loops were completed by the workforce](#c3w3-4.5.)
  - [4.6. View human labels and prepare the data for re-training](#c3w3-4.6.)

Let's install and import the required modules.

In [1]:
# please ignore warning messages during the installation
!pip install --disable-pip-version-check -q sagemaker==2.35.0

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
import boto3
import sagemaker
import pandas as pd
from pprint import pprint
import botocore

config = botocore.config.Config(user_agent_extra='dlai-pds/c3/w3')

# low-level service client of the boto3 session
sm = boto3.client(service_name='sagemaker', 
                  config=config)

sm_runtime = boto3.client('sagemaker-runtime',
                          config=config)

sess = sagemaker.Session(sagemaker_client=sm,
                         sagemaker_runtime_client=sm_runtime)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

s3 = boto3.Session().client(service_name='s3', 
                            config=config)
cognito_idp = boto3.Session().client(service_name='cognito-idp', 
                                     config=config)
a2i = boto3.Session().client(service_name='sagemaker-a2i-runtime', 
                             config=config)

<a name='c3w3-1.'></a>
# 1. Set up Amazon Cognito user pool and define human workforce

The first step in the creation of the human-in-the-loop pipeline will be to create your own private workforce.

<img src="images/human-loop-workflow-1-workforce.png" width="40%" align="center">

Amazon Cognito provides authentication, authorization, and user management for apps. This enables your workers to sign in directly to the labeling UI with a username and password. 

You will construct an Amazon Cognito user pool, setting up its client, domain, and group. Then you'll create a SageMaker workforce, linking it to the Cognito user pool. Followed by the creation of a SageMaker workteam, linking it to the Cognito user pool and group. And finally, you will create a pool user and add it to the group.

To get started, let's construct the user pool and user pool client names.

In [3]:
import time
timestamp = int(time.time())

user_pool_name = 'groundtruth-user-pool-{}'.format(timestamp)
user_pool_client_name = 'groundtruth-user-pool-client-{}'.format(timestamp)

print("Amazon Cognito user pool name: {}".format(user_pool_name))
print("Amazon Cognito user pool client name: {}".format(user_pool_client_name))

Amazon Cognito user pool name: groundtruth-user-pool-1631790671
Amazon Cognito user pool client name: groundtruth-user-pool-client-1631790671


<a name='c3w3-1.1.'></a>
### 1.1. Create Amazon Cognito user pool

Function `cognito_idp.create_user_pool` creates a new Amazon Cognito user pool. Passing the function result into a variable you can get the information about the response. The result is in dictionary format.

In [4]:
create_user_pool_response = cognito_idp.create_user_pool(PoolName=user_pool_name)
user_pool_id = create_user_pool_response['UserPool']['Id']

print("Amazon Cognito user pool ID: {}".format(user_pool_id))

Amazon Cognito user pool ID: us-east-1_rw9sRDG9N


<a name='c3w3-ex-1'></a>
### Exercise 1

Pull the Amazon Cognito user pool name from its description.

**Instructions**: Print the keys of the user pool, choose the one that corresponds to the name and print its value.

In [5]:
print(create_user_pool_response['UserPool'].keys())

dict_keys(['Id', 'Name', 'Policies', 'LambdaConfig', 'LastModifiedDate', 'CreationDate', 'SchemaAttributes', 'VerificationMessageTemplate', 'MfaConfiguration', 'EstimatedNumberOfUsers', 'EmailConfiguration', 'AdminCreateUserConfig', 'Arn'])


In [6]:
### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
user_pool_name = create_user_pool_response['UserPool']['Name'] # Replace all None
### END SOLUTION - DO NOT delete this comment for grading purposes
print('Amazon Cognito user pool name: {}'.format(user_pool_name))

Amazon Cognito user pool name: groundtruth-user-pool-1631790671


<a name='c3w3-1.2.'></a>
### 1.2. Create Amazon Cognito user pool client

Now let's set up the Amazon Cognito user pool client for the created above user pool. 

The Amazon Cognito user pool client implements an open standard for authorization framework, `OAuth`. The standard enables apps to obtain limited access (scopes) to a user’s data without giving away a user’s password. It decouples authentication from authorization and supports multiple use cases addressing different device capabilities.

<a name='c3w3-ex-2'></a>
### Exercise 2

Create the Amazon Cognito user pool client for the constructed user pool.

**Instructions**: Pass the user pool ID and the user pool client name into the function `cognito_idp.create_user_pool_client`. Review the other parameters of the function.

In [7]:
### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
create_user_pool_client_response = cognito_idp.create_user_pool_client( # Replace None
    UserPoolId=user_pool_id, # Replace None
    ClientName=user_pool_client_name, # Replace None
### END SOLUTION - DO NOT delete this comment for grading purposes
    GenerateSecret=True, # boolean to specify whether you want to generate a secret
    # a list of provider names for the identity providers that are supported on this client, e.g. Cognito, Facebook, Google
    SupportedIdentityProviders=[
        'COGNITO' 
    ],
    # a list of the allowed OAuth flows, e.g. code, implicit, client_credentials
    AllowedOAuthFlows=[
        'code',
        'implicit'
    ],
    # a list of the allowed OAuth scopes, e.g. phone, email, openid, and profile
    AllowedOAuthScopes=[
        'email',
        'openid',
        'profile'
    ],
    # a list of allowed redirect (callback) URLs for the identity providers
    CallbackURLs=[
        'https://datascienceonaws.com', 
    ],
    # set to true if the client is allowed to follow the OAuth protocol when interacting with Cognito user pools
    AllowedOAuthFlowsUserPoolClient=True
)

client_id = create_user_pool_client_response['UserPoolClient']['ClientId']
print('Amazon Cognito user pool client ID: {}'.format(client_id))

Amazon Cognito user pool client ID: 7le038ibueq6rau0lu6ipffiok


<a name='c3w3-1.3.'></a>
### 1.3. Create Amazon Cognito user pool domain and group

<a name='c3w3-ex-3'></a>
### Exercise 3

Set up the Amazon Cognito user pool domain for the constructed user pool.

**Instructions**: Pass the user pool ID and the user pool domain name into the function `cognito_idp.create_user_pool_domain`.

In [8]:
user_pool_domain_name = 'groundtruth-user-pool-domain-{}'.format(timestamp)

try:
    ### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
    cognito_idp.create_user_pool_domain( # Replace None  
        UserPoolId=user_pool_id, # Replace None
        Domain=user_pool_domain_name # Replace None
    ### END SOLUTION - DO NOT delete this comment for grading purposes
    )
    print("Created Amazon Cognito user pool domain: {}".format(user_pool_domain_name))
except:
    print("Amazon Cognito user pool domain {} already exists".format(user_pool_domain_name))

Created Amazon Cognito user pool domain: groundtruth-user-pool-domain-1631790671


You will use the following function to check if the Amazon Cognito user group already exists.

In [9]:
def check_user_pool_group_existence(user_pool_id, user_pool_group_name):  
    for group in cognito_idp.list_groups(UserPoolId=user_pool_id)['Groups']:
        if user_pool_group_name == group['GroupName']:
            return True
    return False

<a name='c3w3-ex-4'></a>
### Exercise 4

Set up Amazon Cognito user group.

**Instructions**: Pass the user pool ID and the user group name into the function `cognito_idp.create_group`.

In [10]:
user_pool_group_name = 'groundtruth-user-pool-group-{}'.format(timestamp)

if not check_user_pool_group_existence(user_pool_id, user_pool_group_name):
    ### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
    cognito_idp.create_group( # Replace None
        UserPoolId=user_pool_id, # Replace None
        GroupName=user_pool_group_name # Replace None
    ### END SOLUTION - DO NOT delete this comment for grading purposes
    )
    print("Created Amazon Cognito user group: {}".format(user_pool_group_name))
else:
    print("Amazon Cognito user group {} already exists".format(user_pool_group_name))

Created Amazon Cognito user group: groundtruth-user-pool-group-1631790671


<a name='c3w3-1.4.'></a>
### 1.4. Create workforce and workteam

Use the following function to check if the workforce already exists. You can only create one workforce per region, therefore you'll have to delete any other existing workforce, together with all of the related workteams.

In [11]:
def check_workforce_existence(workforce_name):  
    for workforce in sm.list_workforces()['Workforces']:
        if workforce_name == workforce['WorkforceName']:
            return True
        else:
            for workteam in sm.list_workteams()['Workteams']:
                sm.delete_workteam(WorkteamName=workteam['WorkteamName'])
            sm.delete_workforce(WorkforceName=workforce['WorkforceName'])
    return False

<a name='c3w3-ex-5'></a>
### Exercise 5

Create a workforce.

**Instructions**: Pass the Amazon Cognito user pool ID and client ID into the Cognito configuration of the function `sm.create_workforce`.

In [12]:
workforce_name = 'groundtruth-workforce-name-{}'.format(timestamp)

if not check_workforce_existence(workforce_name):
    create_workforce_response = sm.create_workforce(
        WorkforceName=workforce_name,
        CognitoConfig={
            ### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
            'UserPool': user_pool_id, # Replace None
            'ClientId': client_id # Replace None
            ### END SOLUTION - DO NOT delete this comment for grading purposes
        }
    )
    print("Workforce name: {}".format(workforce_name))
    pprint(create_workforce_response)
else:
    print("Workforce {} already exists".format(workforce_name))

Workforce name: groundtruth-workforce-name-1631790671
{'ResponseMetadata': {'HTTPHeaders': {'content-length': '107',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Thu, 16 Sep 2021 11:11:22 GMT',
                                      'x-amzn-requestid': 'f22c98c6-031c-424b-9464-072878f2890b'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'f22c98c6-031c-424b-9464-072878f2890b',
                      'RetryAttempts': 0},
 'WorkforceArn': 'arn:aws:sagemaker:us-east-1:851223673325:workforce/groundtruth-workforce-name-1631790671'}


You can use the `sm.describe_workforce` function to get the information about the workforce.

In [13]:
describe_workforce_response = sm.describe_workforce(WorkforceName=workforce_name)
describe_workforce_response

{'Workforce': {'WorkforceName': 'groundtruth-workforce-name-1631790671',
  'WorkforceArn': 'arn:aws:sagemaker:us-east-1:851223673325:workforce/groundtruth-workforce-name-1631790671',
  'LastUpdatedDate': datetime.datetime(2021, 9, 16, 11, 11, 22, 680000, tzinfo=tzlocal()),
  'SourceIpConfig': {'Cidrs': []},
  'SubDomain': 'dxqlr8tnmi.labeling.us-east-1.sagemaker.aws',
  'CognitoConfig': {'UserPool': 'us-east-1_rw9sRDG9N',
   'ClientId': '7le038ibueq6rau0lu6ipffiok'},
  'CreateDate': datetime.datetime(2021, 9, 16, 11, 11, 22, 680000, tzinfo=tzlocal())},
 'ResponseMetadata': {'RequestId': '23d9f53b-71ac-42e3-a14a-31fb413e6207',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '23d9f53b-71ac-42e3-a14a-31fb413e6207',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '443',
   'date': 'Thu, 16 Sep 2021 11:11:22 GMT'},
  'RetryAttempts': 0}}

Use the following function to check if the workteam already exists. If there are no workteams in the list, give some time for the workforce to set up.

In [14]:
def check_workteam_existence(workteam_name):  
    if sm.list_workteams()['Workteams']:
        for workteam in sm.list_workteams()['Workteams']:
            if workteam_name == workteam['WorkteamName']:
                return True
    else:
        time.sleep(60)
        return False
    return False

<a name='c3w3-ex-6'></a>
### Exercise 6

Create a workteam.

**Instructions**: Pass the Amazon Cognito user pool ID, client ID, and group name into the Cognito member definition of the function `sm.create_workteam`.

### _This cell may take 1-2 minutes to run._

In [15]:
workteam_name = 'groundtruth-workteam-{}'.format(timestamp)

if not check_workteam_existence(workteam_name):
    create_workteam_response = sm.create_workteam(
        Description='groundtruth workteam',
        WorkforceName=workforce_name,
        WorkteamName=workteam_name,
        # objects that identify the workers that make up the work team
        MemberDefinitions=[{
            'CognitoMemberDefinition': {
                ### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
                'UserPool': user_pool_id, # Replace None
                'ClientId': client_id, # Replace None
                'UserGroup': user_pool_group_name # Replace None
                ### END SOLUTION - DO NOT delete this comment for grading purposes
            }
        }]
    )
    pprint(create_workteam_response)
else:
    print("Workteam {} already exists".format(workteam_name))

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '113',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Thu, 16 Sep 2021 11:12:25 GMT',
                                      'x-amzn-requestid': '301b83c9-a1c5-4508-b6fa-9ba878c993a4'},
                      'HTTPStatusCode': 200,
                      'RequestId': '301b83c9-a1c5-4508-b6fa-9ba878c993a4',
                      'RetryAttempts': 0},
 'WorkteamArn': 'arn:aws:sagemaker:us-east-1:851223673325:workteam/private-crowd/groundtruth-workteam-1631790671'}


You can use `sm.describe_workteam` function to get information about the workteam.

In [16]:
describe_workteam_response = sm.describe_workteam(WorkteamName=workteam_name)
describe_workteam_response

{'Workteam': {'WorkteamName': 'groundtruth-workteam-1631790671',
  'MemberDefinitions': [{'CognitoMemberDefinition': {'UserPool': 'us-east-1_rw9sRDG9N',
     'UserGroup': 'groundtruth-user-pool-group-1631790671',
     'ClientId': '7le038ibueq6rau0lu6ipffiok'}}],
  'WorkteamArn': 'arn:aws:sagemaker:us-east-1:851223673325:workteam/private-crowd/groundtruth-workteam-1631790671',
  'Description': 'groundtruth workteam',
  'SubDomain': 'dxqlr8tnmi.labeling.us-east-1.sagemaker.aws',
  'CreateDate': datetime.datetime(2021, 9, 16, 11, 12, 25, 738000, tzinfo=tzlocal()),
  'LastUpdatedDate': datetime.datetime(2021, 9, 16, 11, 12, 25, 738000, tzinfo=tzlocal()),
  'NotificationConfiguration': {}},
 'ResponseMetadata': {'RequestId': '5dcc42b9-5f96-4e06-96ca-ee76ae166f1c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5dcc42b9-5f96-4e06-96ca-ee76ae166f1c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '544',
   'date': 'Thu, 16 Sep 2021 11:12:25 GMT'},
  'Re

Now you can pull the workteam ARN either from `create_workteam_response` or `describe_workteam_response`.

In [17]:
workteam_arn = describe_workteam_response['Workteam']['WorkteamArn']
workteam_arn

'arn:aws:sagemaker:us-east-1:851223673325:workteam/private-crowd/groundtruth-workteam-1631790671'

Review the created workteam in the AWS console.

**Instructions**:

- open the link
- notice that you are in the section Amazon SageMaker -> Labeling workforces
- check the name of the workteam, its Amazon Cognito user group and ARN

In [18]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://{}.console.aws.amazon.com/sagemaker/groundtruth?region={}#/labeling-workforces/private-details/{}">workteam</a></b>'.format(region, region, workteam_name)))

<a name='c3w3-1.5.'></a>
### 1.5. Create an Amazon Cognito user and add the user to the group

Use the following function to check if the Amazon Cognito user already exists.

In [19]:
def check_user_existence(user_pool_id, user_name):  
    for user in cognito_idp.list_users(UserPoolId=user_pool_id)['Users']:
        if user_name == user['Username']:
            return True
    return False

Create a user passing the username, temporary password, and the Amazon Cognito user pool ID.

In [20]:
user_name = 'user-{}'.format(timestamp)

temporary_password = 'Password@420'

if not check_user_existence(user_pool_id, user_name):
    create_user_response=cognito_idp.admin_create_user(
        Username=user_name,
        UserPoolId=user_pool_id,
        TemporaryPassword=temporary_password,
        MessageAction='SUPPRESS' # suppress sending the invitation message to a user that already exists
    )
    pprint(create_user_response)
else:
    print("Amazon Cognito user {} already exists".format(user_name))

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '242',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Thu, 16 Sep 2021 11:12:45 GMT',
                                      'x-amzn-requestid': '301757b1-63f7-47db-8bcc-8b1709a51a75'},
                      'HTTPStatusCode': 200,
                      'RequestId': '301757b1-63f7-47db-8bcc-8b1709a51a75',
                      'RetryAttempts': 0},
 'User': {'Attributes': [{'Name': 'sub',
                          'Value': 'f145d327-7efa-45ad-9c97-53ecddecb963'}],
          'Enabled': True,
          'UserCreateDate': datetime.datetime(2021, 9, 16, 11, 12, 45, 779000, tzinfo=tzlocal()),
          'UserLastModifiedDate': datetime.datetime(2021, 9, 16, 11, 12, 45, 779000, tzinfo=tzlocal()),
          'UserStatus': 'FORCE_CHANGE_PASSWORD',
          'Username': 'user-1631790671'}}


Add the user into the Amazon Cognito user group.

In [21]:
cognito_idp.admin_add_user_to_group(
    UserPoolId=user_pool_id,
    Username=user_name,
    GroupName=user_pool_group_name
)

{'ResponseMetadata': {'RequestId': '0372ed46-09d9-45a1-8bbc-fd30cb6ae88a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 16 Sep 2021 11:12:46 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0372ed46-09d9-45a1-8bbc-fd30cb6ae88a'},
  'RetryAttempts': 0}}

<a name='c3w3-2.'></a>
# 2. Create Human Task UI

<img src="images/human-loop-workflow-2-taskUI.png" width="40%" align="center">

Create a Human Task UI resource, using a worker task UI template.  This template will be rendered to the human workers whenever human interaction is required.

Below there is a simple demo template provided, that is compatible with the current use case of classifying product reviews into the three sentiment classes. For other pre-built UIs (there are 70+), check: https://github.com/aws-samples/amazon-a2i-sample-task-uis

In [22]:
template = r"""
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
    <crowd-classifier name="sentiment"
                      categories="['-1', '0', '1']"
                      initial-value="{{ task.input.initialValue }}"
                      header="Classify Reviews into Sentiment:  -1 (negative), 0 (neutral), and 1 (positive)">
      
        <classification-target>
            {{ task.input.taskObject }}
        </classification-target>
      
        <full-instructions header="Classify reviews into sentiment:  -1 (negative), 0 (neutral), and 1 (positive)">
            <p><strong>1</strong>: joy, excitement, delight</p>       
            <p><strong>0</strong>: neither positive or negative, such as stating a fact</p>
            <p><strong>-1</strong>: anger, sarcasm, anxiety</p>
        </full-instructions>

        <short-instructions>
            Classify reviews into sentiment:  -1 (negative), 0 (neutral), and 1 (positive)
        </short-instructions>
    </crowd-classifier>
</crowd-form>
"""

<a name='c3w3-ex-7'></a>
### Exercise 7

Create a human task UI resource.

**Instructions**: Pass the worker task UI template defined above as the content of the UI template parameter to the function `sm.create_human_task_ui`.

In [23]:
# Task UI name - this value is unique per account and region. You can also provide your own value here.
task_ui_name = 'ui-{}'.format(timestamp)

human_task_ui_response = sm.create_human_task_ui(
    HumanTaskUiName=task_ui_name,
    UiTemplate={
        ### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
        "Content": template  # Replace None
        ### END SOLUTION - DO NOT delete this comment for grading purposes
    }
)
human_task_ui_response

{'HumanTaskUiArn': 'arn:aws:sagemaker:us-east-1:851223673325:human-task-ui/ui-1631790671',
 'ResponseMetadata': {'RequestId': '570a22d8-6fbc-431f-ac46-5cddad1d2da6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '570a22d8-6fbc-431f-ac46-5cddad1d2da6',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '89',
   'date': 'Thu, 16 Sep 2021 11:12:48 GMT'},
  'RetryAttempts': 0}}

Pull the ARN of the human task UI:

In [24]:
human_task_ui_arn = human_task_ui_response["HumanTaskUiArn"]
print(human_task_ui_arn)

arn:aws:sagemaker:us-east-1:851223673325:human-task-ui/ui-1631790671


<a name='c3w3-3.'></a>
# 3. Define human review workflow

<img src="images/human-loop-workflow-3-review.png" width="40%" align="center">

In this section, you are going to create a Flow Definition. Flow Definitions allow you to specify:

- The workforce (in fact, it is a workteam) that your tasks will be sent to.
- The instructions that your workforce will receive (worker task template).
- The configuration of your worker tasks, including the number of workers that receive a task and time limits to complete tasks.
- Where your output data will be stored.

Here you are going to use the API, but you can optionally create this workflow definition in the console as well. 

For more details and instructions, see: https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html.

Let's construct the S3 bucket output path.

In [25]:
output_path = 's3://{}/a2i-results-{}'.format(bucket, timestamp)
print(output_path)

s3://sagemaker-us-east-1-851223673325/a2i-results-1631790671


<a name='c3w3-ex-8'></a>
### Exercise 8

Construct the Flow Definition with the workteam and human task UI in the human loop configurations that you created above.

**Instructions**: Pass the workteam and human task UI ARNs into the `HumanLoopConfig` dictionary within the function `sm.create_flow_definition`. Review the other parameters.

In [26]:
# Flow definition name - this value is unique per account and region
flow_definition_name = 'fd-{}'.format(timestamp)

create_workflow_definition_response = sm.create_flow_definition(
    FlowDefinitionName=flow_definition_name,
    RoleArn=role,
    HumanLoopConfig={
        ### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
        "WorkteamArn": workteam_arn, # Replace None
        "HumanTaskUiArn": human_task_ui_arn, # Replace None
        ### END SOLUTION - DO NOT delete this comment for grading purposes
        "TaskCount": 1, # the number of workers that receive a task
        "TaskDescription": "Classify Reviews into sentiment:  -1 (negative), 0 (neutral), 1 (positive)",
        "TaskTitle": "Classify Reviews into sentiment:  -1 (negative), 0 (neutral), 1 (positive)",
    },
    OutputConfig={"S3OutputPath": output_path},
)

augmented_ai_flow_definition_arn = create_workflow_definition_response["FlowDefinitionArn"]

You can pull information about the Flow Definition with the function `sm.describe_flow_definition` and wait for its status value `FlowDefinitionStatus` to become `Active`.

In [27]:
for _ in range(60):
    describe_flow_definition_response = sm.describe_flow_definition(FlowDefinitionName=flow_definition_name)
    print(describe_flow_definition_response["FlowDefinitionStatus"])
    if describe_flow_definition_response["FlowDefinitionStatus"] == "Active":
        print("Flow Definition is active")
        break
    time.sleep(2)

Initializing
Active
Flow Definition is active


<a name='c3w3-4.'></a>
# 4. Start human loop with custom ML model

<img src="images/human-loop-workflow-4-start.png" width="40%" align="center">

Deploy a custom ML model into an endpoint and call it to predict labels for some sample reviews. Check the confidence score for each prediction. If it is smaller than the threshold, engage your workforce for a human review, starting a human loop. Fix the labels by completing the human loop tasks and review the results.

<img src="images/augmented-ai-loop.png" width="60%" align="center">

<a name='c3w3-4.1.'></a>
### 4.1. Deploy a custom model

Set up a sentiment predictor class to be wrapped later into the PyTorch Model.

<a name='c3w3-ex-9'></a>
### Exercise 9

Create a Sentiment Predictor class.

**Instructions**: Pass the JSON serializer and deserializer objects here, calling them with the functions `JSONLinesSerializer()` and `JSONLinesDeserializer()`, respectively. More information about the serializers can be found [here](https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html).

In [28]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

class SentimentPredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(
            endpoint_name, 
            sagemaker_session=sagemaker_session,
            ### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
            serializer=JSONLinesSerializer(), # Replace None
            deserializer=JSONLinesDeserializer() # Replace None
            ### END SOLUTION - DO NOT delete this comment for grading purposes
        )

Create a SageMaker model based on the model artifact saved in the S3 bucket.

In [29]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model_name = 'model-{}'.format(timestamp)

model = PyTorchModel(name=pytorch_model_name,
                     model_data='s3://dlai-practical-data-science/models/ab/variant_a/model.tar.gz',
                     predictor_cls=SentimentPredictor,
                     entry_point='inference.py',
                     source_dir='src',
                     framework_version='1.6.0',
                     py_version='py3',
                     role=role)

Now you will create a SageMaker Endpoint from the model. For the purposes of this lab, you will use a relatively small instance type. Please refer to [this link](https://aws.amazon.com/sagemaker/pricing/) for additional instance types that may work for your use cases outside of this lab.

### _This cell will take approximately 5-10 minutes to run._

In [30]:
%%time

pytorch_endpoint_name = 'endpoint-{}'.format(timestamp)

predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m5.large', 
                         endpoint_name=pytorch_endpoint_name)

-----------------!CPU times: user 2min 15s, sys: 10.8 s, total: 2min 25s
Wall time: 10min 53s


You can review the endpoint in the AWS console and check its status.

In [31]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(region, pytorch_endpoint_name)))

<a name='c3w3-4.2.'></a>
### 4.2. Start the human loop

Let's create a list of sample reviews.

In [32]:
reviews = ["I enjoy this product", 
           "I am unhappy with this product", 
           "It is okay", 
           "sometimes it works"]

Now you can send each of the sample reviews to the model via the `predictor.predict()` API call. Note that you need to pass the reviews in the JSON format that model expects as input. Then, you parse the model's response to obtain the predicted label and the confidence score. 

After that, you check the condition for when you want to engage a human for review. You can check whether the returned confidence score is under the defined threshold of 90%, which would mean that you would want to start the human loop with the predicted label and the review as inputs. Finally, you start the human loop passing the input content and Flow Definition defined above.

<a name='c3w3-ex-10'></a>
### Exercise 10

Complete the dictionary `input_content`, which should contain the original prediction (`'initialValue'` key) and review text (`'taskObject'` key).

In [41]:
import json

human_loops_started = []

CONFIDENCE_SCORE_THRESHOLD = 0.90

for review in reviews:
    inputs = [
        {"features": [review]},
    ]

    response = predictor.predict(inputs)
    print(response)
    prediction = response[0]['predicted_label']
    confidence_score = response[0]['probability']

    print('Checking prediction confidence {} for sample review: "{}"'.format(confidence_score, review))

    # condition for when you want to engage a human for review
    if confidence_score < CONFIDENCE_SCORE_THRESHOLD:
        human_loop_name = str(time.time()).replace('.', '-') # using milliseconds
        input_content = {
            ### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
            "initialValue": prediction, # Replace None
            "taskObject": review # Replace None
            ### END SOLUTION - DO NOT delete this comment for grading purposes
        }
        start_loop_response = a2i.start_human_loop(
            HumanLoopName=human_loop_name,
            FlowDefinitionArn=augmented_ai_flow_definition_arn,
            HumanLoopInput={"InputContent": json.dumps(input_content)},
        )

        human_loops_started.append(human_loop_name)

        print(
            f"Confidence score of {confidence_score * 100}% for prediction of {prediction} is less than the threshold of {CONFIDENCE_SCORE_THRESHOLD * 100}%"
        )
        print(f"*** ==> Starting human loop with name: {human_loop_name}  \n")
    else:
        print(
            f"Confidence score of {confidence_score * 100}% for star rating of {prediction} is above threshold of {CONFIDENCE_SCORE_THRESHOLD * 100}%"
        )
        print("Human loop not needed. \n")

[{'probability': 0.9376369118690491, 'predicted_label': 1}]
Checking prediction confidence 0.9376369118690491 for sample review: "I enjoy this product"
Confidence score of 93.76369118690491% for star rating of 1 is above threshold of 90.0%
Human loop not needed. 

[{'probability': 0.6340296864509583, 'predicted_label': -1}]
Checking prediction confidence 0.6340296864509583 for sample review: "I am unhappy with this product"
Confidence score of 63.402968645095825% for prediction of -1 is less than the threshold of 90.0%
*** ==> Starting human loop with name: 1631792290-4894407  

[{'probability': 0.5422114729881287, 'predicted_label': 1}]
Checking prediction confidence 0.5422114729881287 for sample review: "It is okay"
Confidence score of 54.221147298812866% for prediction of 1 is less than the threshold of 90.0%
*** ==> Starting human loop with name: 1631792291-0051627  

[{'probability': 0.3931102454662323, 'predicted_label': 1}]
Checking prediction confidence 0.3931102454662323 for s

Review the results above. Three of the sample reviews with the probability scores lower than the threshold went into the human loop. The original predicted labels are passed together with the review text and will be seen in the task.

<a name='c3w3-4.3.'></a>
### 4.3. Check status of the human loop

Function `a2i.describe_human_loop` can be used to pull the information about the human loop.

In [42]:
completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f"HumanLoop Name: {human_loop_name}")
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print("")

    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)

HumanLoop Name: 1631792290-4894407
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-851223673325/a2i-results-1631790671/fd-1631790671/2021/09/16/11/38/10/1631792290-4894407/output.json'}

HumanLoop Name: 1631792291-0051627
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-851223673325/a2i-results-1631790671/fd-1631790671/2021/09/16/11/38/11/1631792291-0051627/output.json'}

HumanLoop Name: 1631792291-5234158
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-851223673325/a2i-results-1631790671/fd-1631790671/2021/09/16/11/38/11/1631792291-5234158/output.json'}



<a name='c3w3-4.4.'></a>
### 4.4. Complete the human loop tasks

Pull labeling UI from the workteam information to get into the human loop tasks in the AWS console.

In [43]:
labeling_ui = sm.describe_workteam(WorkteamName=workteam_name)["Workteam"]["SubDomain"]
print(labeling_ui)

dxqlr8tnmi.labeling.us-east-1.sagemaker.aws


Navigate to the link below and login with the defined username and password. Complete the human loop following the provided instructions.

In [44]:
from IPython.core.display import display, HTML

display(HTML('Click <a target="blank" href="https://{}"><b>here</b></a> to start labeling with username <b>{}</b> and temporary password <b>{}</b>'.format(labeling_ui, user_name, temporary_password)))

<img src="images/label-data-job-instructions.png" width="80%" align="center">

_Wait for workers to complete ^^ their human loop tasks ^^_

<a name='c3w3-4.5.'></a>
### 4.5. Verify that the human loops were completed by the workforce

## _Note:  This cell will not complete until you label the data following the instructions above._

In [45]:
import time

completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f"HumanLoop Name: {human_loop_name}")
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print("")
    while resp["HumanLoopStatus"] != "Completed":
        print(f"Waiting for HumanLoop to complete.")
        time.sleep(10)
        resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)
        print(f"Completed!")
        print("")

HumanLoop Name: 1631792290-4894407
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-851223673325/a2i-results-1631790671/fd-1631790671/2021/09/16/11/38/10/1631792290-4894407/output.json'}

Completed!

HumanLoop Name: 1631792291-0051627
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-851223673325/a2i-results-1631790671/fd-1631790671/2021/09/16/11/38/11/1631792291-0051627/output.json'}

Completed!

HumanLoop Name: 1631792291-5234158
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-851223673325/a2i-results-1631790671/fd-1631790671/2021/09/16/11/38/11/1631792291-5234158/output.json'}

Completed!



## _Note:  This cell ^^ above ^^ will not complete until you label the data following the instructions above._

<a name='c3w3-4.6.'></a>
### 4.6. View human labels and prepare the data for re-training

Once the work is complete, Amazon A2I stores the results in the specified S3 bucket and sends a Cloudwatch Event.  Let's check the S3 contents.

In [46]:
import re
from pprint import pprint

fixed_items = []

for resp in completed_human_loops:
    split_string = re.split("s3://" + bucket + "/", resp["HumanLoopOutput"]["OutputS3Uri"])
    output_bucket_key = split_string[1]

    response = s3.get_object(Bucket=bucket, Key=output_bucket_key)
    content = response["Body"].read().decode("utf-8")
    json_output = json.loads(content)
    pprint(json_output)

    input_content = json_output["inputContent"]
    human_answer = json_output["humanAnswers"][0]["answerContent"]
    fixed_item = {"input_content": input_content, "human_answer": human_answer}
    fixed_items.append(fixed_item)

{'flowDefinitionArn': 'arn:aws:sagemaker:us-east-1:851223673325:flow-definition/fd-1631790671',
 'humanAnswers': [{'acceptanceTime': '2021-09-16T11:38:47.892Z',
                   'answerContent': {'sentiment': {'label': '-1'}},
                   'submissionTime': '2021-09-16T11:38:51.184Z',
                   'timeSpentInSeconds': 3.292,
                   'workerId': 'b2f4b452471f9dfe',
                   'workerMetadata': {'identityData': {'identityProviderType': 'Cognito',
                                                       'issuer': 'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_rw9sRDG9N',
                                                       'sub': 'f145d327-7efa-45ad-9c97-53ecddecb963'}}}],
 'humanLoopName': '1631792290-4894407',
 'inputContent': {'initialValue': -1,
                  'taskObject': 'I am unhappy with this product'}}
{'flowDefinitionArn': 'arn:aws:sagemaker:us-east-1:851223673325:flow-definition/fd-1631790671',
 'humanAnswers': [{'acceptanceTime': '

Now you can prepare the data for re-training.

In [47]:
df_fixed_items = pd.DataFrame(fixed_items)  
df_fixed_items.head()

,input_content,human_answer
0,"{'initialValue': -1, 'taskObject': 'I am unhap...",{'sentiment': {'label': '-1'}}
1,"{'initialValue': 1, 'taskObject': 'It is okay'}",{'sentiment': {'label': '1'}}
2,"{'initialValue': 1, 'taskObject': 'sometimes i...",{'sentiment': {'label': '0'}}


Upload the notebook into S3 bucket for grading purposes.

**Note:** you may need to click on "Save" button before the upload.

In [48]:
!aws s3 cp ./C3_W3_Assignment.ipynb s3://$bucket/C3_W3_Assignment_Learner.ipynb

upload: ./C3_W3_Assignment.ipynb to s3://sagemaker-us-east-1-851223673325/C3_W3_Assignment_Learner.ipynb
